## Libs

In [204]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
from joblib import Parallel, delayed


import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Load Data

In [205]:

# dataset de vendas
sales = pd.read_csv('static_data/sales_train_evaluation.csv')

# Converter colunas de vendas para int16
d_cols = [col for col in sales.columns if col.startswith('d_')]
sales[d_cols] = sales[d_cols].astype(np.int16)

# dataset alendário
calendar = pd.read_csv('static_data/calendar.csv')

# dataset de preços
sell_prices = pd.read_csv('static_data/sell_prices.csv')

print('Dimensões M5 Base - Sales',sales.shape)
print ('Dimensões M5 Base - Calendar', calendar.shape)
print ('Dimensões M5 Base - Sell prices', sell_prices.shape)

Dimensões M5 Base - Sales (30490, 1947)
Dimensões M5 Base - Calendar (1969, 14)
Dimensões M5 Base - Sell prices (6841121, 4)


## Pre-prossing

#### 1 - Data Types

In [206]:
calendar.dtypes

date            object
wm_yr_wk         int64
weekday         object
wday             int64
month            int64
year             int64
d               object
event_name_1    object
event_type_1    object
event_name_2    object
event_type_2    object
snap_CA          int64
snap_TX          int64
snap_WI          int64
dtype: object

In [207]:
sell_prices.dtypes

store_id       object
item_id        object
wm_yr_wk        int64
sell_price    float64
dtype: object

- Para a base `sales` temos um alto volume de linhas e colunas, fazer a abordagem tradicional de avaliação de tipagem seria bem complexa. 
- Para essa base seguirei uma abordagem de interpretação lógica. A `base original` contém `1947 colunas`, sendo que 1941 são do tipo 'int' e apenas 6 'object'. Ao avaliar as top 10 colunas, chegamos a conclusão que as 6 primeiras são do tipo 'object', com isso, chegamos a conclusão que todas as outras são 'int'. Importante essa analise, no dia a dia, uma base que é de gestão do time de negócio pode vir colunas com a mesma informação (exemplo: vendas/mês), porém com tipagem diferentes. Caso não tratado pode gerar problemas de conversão ou até `overfitting` no modelo. 

In [208]:
print('\nBase em seu total de colunas:\n',sales.dtypes.value_counts())
print('-'*60)

sales_top10_cols = sales.dtypes[:10]
dtype_counts = sales_top10_cols.value_counts()
print('\nBase apenas com TOP 10 colunas:\n',dtype_counts)


Base em seu total de colunas:
 int16     1941
object       6
Name: count, dtype: int64
------------------------------------------------------------

Base apenas com TOP 10 colunas:
 object    6
int16     4
Name: count, dtype: int64


#### 2 - Missing Values

- A única base com valores ausentes foi a `calendario`, segui a seguinte abordagem:
    - Preenchi os valores nulos das colunas de eventos com 'No_Event' para representar explicitamente a ausência de eventos nos dias correspondentes. Valores nulos poderiam causar problemas durante o processamento e modelagem dos dados. Além disso, facilita a codificação das variáveis categóricas e permite que o modelo diferencie entre dias com e sem eventos, potencialmente melhorando a precisão das previsões.

In [209]:
sales_na = sales.isna().sum()>0
sales_na.value_counts()

False    1947
Name: count, dtype: int64

In [210]:
sell_prices_na = sell_prices.isna().sum()>1
sell_prices_na.value_counts()

False    4
Name: count, dtype: int64

In [211]:
calendar_na = calendar.isna().sum()>0
calendar_na.value_counts()

False    10
True      4
Name: count, dtype: int64

In [212]:
# eventos
event_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

# adicionar 'No_Event' em valores nulos 
calendar[event_cols] = calendar[event_cols].fillna('No_Event')


## EDA

In [213]:
sales['cat_id'].unique()

array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object)

- Escolha de uma Categoria (FOODS)

In [214]:
categoria_escolhida = 'FOODS'
sales_cat = sales[sales['cat_id'] == categoria_escolhida]

In [215]:
# Verificar o número de itens únicos na categoria
num_itens = sales_cat['item_id'].nunique()
print(f"Número de itens na categoria '{categoria_escolhida}': {num_itens}")

Número de itens na categoria 'FOODS': 1437


- sales_001 --> Dataframe com a categoria já filtrada, no qual será utilizado para acrescemo de features e treinamento de modelos.

In [216]:
# Dataframe, formato wide p/long
sales_001 = pd.melt(sales_cat,
                     id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                     var_name='d',
                     value_name='sales')


In [217]:
# Atribuindo a base de calendario
sales_001 = sales_001.merge(calendar[['d', 'date']], on='d', how='left')

# Convertendo a coluna 'date' para datetime
sales_001['date'] = pd.to_datetime(sales_001['date'])

# adicionando coluna para o mês
sales_001['month'] = sales_001['date'].dt.to_period('M')



In [218]:
sales_001 = sales_001.astype({
    'id': 'category',
    'item_id': 'category',
    'dept_id': 'category',
    'cat_id': 'category',
    'store_id': 'category',
    'state_id': 'category'
})

- Dataset contém 5 anos de dados, histórico suficiente para avaliarmos a nivel de uma categoria unica 

In [219]:
print('Registro mais antigo /',sales_001['date'].min())
print('Registro mais mais novo /',sales_001['date'].max())

Registro mais antigo / 2011-01-29 00:00:00
Registro mais mais novo / 2016-05-22 00:00:00


In [220]:
#sales_001['sell_price'] = sales_001['sell_price'].fillna(0)

- Agregação + Pivot Table (Múltiplas Séries Temporais)

In [221]:
# Agregaando sales na granularidade item/estado/mês
agg_sales = sales_001.groupby(['item_id', 'state_id', 'month'])['sales'].sum().reset_index()


In [222]:
# Criar uma tabela pivot
pivot_sales = agg_sales.pivot_table(
    index='month', 
    columns=['state_id', 'item_id'], 
    values='sales'
)

# garantindo que valores ausentes sejam preenchidos com zeros
pivot_sales = pivot_sales.fillna(0)


In [223]:
# Função para processar cada série temporal individualmente e gerar previsão real
def process_series(state, item, series):
    series = series.dropna()

    # Verificar se a série tem dados suficientes
    if len(series) < 12:
        return None  # Pular se não houver dados suficientes
    
    # Dividir em treinamento e teste
    train = series[:-1]  # Usar todos os meses menos o último para treinamento
    test = series[-1:]   # Usar o último mês para teste (horizonte de 1 mês)
    
    # Verificar se o conjunto de treinamento tem dados suficientes
    if len(train) < 6:
        return None  # Pular se não houver dados suficientes para treinar o modelo
    
    try:
        # Ajustar o modelo de Exponential Smoothing com previsão real
        model = ExponentialSmoothing(train,
                                     trend='add',
                                     seasonal='mul',
                                     seasonal_periods=12).fit()
        # Fazer a previsão de 30 dias (horizonte de 1 mês)
        forecast = model.forecast(steps=30)
        
        # Limitar previsões a valores positivos
        #forecast = np.maximum(forecast, 0)

        
        # Avaliar o desempenho no último mês
        mae = mean_absolute_error(test, model.forecast(steps=1))
        rmse = np.sqrt(mean_squared_error(test, model.forecast(steps=1)))
        
        # Retornar os resultados e previsões reais
        return {
            'state_id': state,
            'item_id': item,
            'MAE': mae,
            'RMSE': rmse,
            'forecast': forecast.tolist()  # Armazenar a previsão real
        }
    except Exception as e:
        print(f"Erro ao processar {item} em {state}: {e}")
        return None

In [224]:
# método joblib -  paralelizar o processamento de cada série temporal
results = Parallel(n_jobs=-1)(delayed(process_series)(state, item, pivot_sales[(state, item)]) 
                              for state, item in pivot_sales.columns)


In [225]:
# Remover resultados nulos
results = [r for r in results if r is not None]

In [226]:
# Converter os resultados em DataFrame
results_df = pd.DataFrame(results)

In [227]:
# Exibir os primeiros resultados
print(results_df.head())

  state_id      item_id         MAE        RMSE                                           forecast
0       CA  FOODS_1_002    6.868356    6.868356  [62.86835582783163, 57.740148378993524, 51.494...
1       CA  FOODS_1_003    7.183433    7.183433  [77.81656677042074, 87.26845567830868, 82.6082...
2       CA  FOODS_1_006   61.577213   61.577213  [185.57721293339995, 196.38065680968248, 167.2...
3       CA  FOODS_1_016   63.374604   63.374604  [113.37460350397076, 107.80862407632968, 107.1...
4       CA  FOODS_1_018  354.543383  354.543383  [1282.5433830097081, 1131.9593411205497, 1392....


In [228]:
# Estrutura da tabela de previsões para 1 mês
forecast_days = 30
columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] + [f'd_{i}' for i in range(1, forecast_days + 1)]

# Inicializar a tabela de previsão
forecast_table = pd.DataFrame(columns=columns)

In [229]:

# Gerar previsões reais para cada item e adicionar à tabela
for result in results_df.itertuples():
    id_value = f"{result.item_id}_{result.state_id}_validation"
    
    # Obter os valores corretos de dept_id, cat_id, e store_id para cada item_id e state_id
    dept_id_value = sales_cat.loc[(sales_cat['item_id'] == result.item_id) & (sales_cat['state_id'] == result.state_id), 'dept_id'].values[0]
    cat_id_value = sales_cat.loc[(sales_cat['item_id'] == result.item_id) & (sales_cat['state_id'] == result.state_id), 'cat_id'].values[0]
    store_id_value = sales_cat.loc[(sales_cat['item_id'] == result.item_id) & (sales_cat['state_id'] == result.state_id), 'store_id'].values[0]

    # Usar as previsões reais geradas na etapa anterior
    forecast_values = result.forecast
    
    # Adicionar à tabela
    forecast_row = [id_value, result.item_id, dept_id_value, cat_id_value, store_id_value, result.state_id] + forecast_values
    forecast_table.loc[len(forecast_table)] = forecast_row

In [230]:
forecast_table

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30
0,FOODS_1_002_CA_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,62.868356,57.740148,51.494747,62.254515,50.158350,53.887152,46.042449,50.768834,42.916480,47.995689,55.802877,59.021301,63.358864,58.190353,51.895995,62.739289,50.548677,54.306225,46.400284,51.163145,43.249588,48.367980,56.235446,59.478524,63.849372,58.640557,52.297243,63.224062,50.939005,54.725298
1,FOODS_1_003_CA_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,77.816567,87.268456,82.608254,79.515628,86.134726,91.846135,77.376775,74.950089,66.650013,66.267355,84.639735,81.433735,78.020737,87.497375,82.824901,79.724119,86.360523,92.086851,77.579525,75.146438,66.824580,66.440881,84.861322,81.646882,78.224906,87.726294,83.041549,79.932610,86.586319,92.327568
2,FOODS_1_006_CA_validation,FOODS_1_006,FOODS_1,FOODS,CA_1,CA,185.577213,196.380657,167.248242,188.395226,135.962994,167.651246,118.265092,140.452129,97.645964,183.105000,157.621941,149.377322,173.069956,183.070531,155.848241,175.480442,126.588970,156.025669,110.016474,130.598764,90.755365,170.107365,146.366631,138.646884,160.562700,169.760405,144.448239,162.565658,117.214947,144.400092
3,FOODS_1_016_CA_validation,FOODS_1_016,FOODS_1,FOODS,CA_1,CA,113.374604,107.808624,107.153647,102.467813,85.661280,71.884580,74.677840,81.779024,81.219957,86.734757,88.422764,80.358655,85.800857,81.046168,79.991896,75.933341,62.989689,52.430130,54.001118,58.601349,57.643908,60.933791,61.451063,55.207433,58.227110,54.283713,52.830145,49.398869,40.318097,32.975679
4,FOODS_1_018_CA_validation,FOODS_1_018,FOODS_1,FOODS,CA_1,CA,1282.543383,1131.959341,1392.996751,1461.883812,1243.865861,1390.488921,975.317095,1471.682460,1230.112600,1283.413136,1621.837912,1499.332907,1496.204872,1317.952647,1618.789673,1695.684646,1440.182395,1607.097733,1125.303839,1695.137904,1414.555107,1473.472713,1859.086699,1716.019700,1709.866361,1503.945954,1844.582595,1929.485481,1636.498928,1823.706545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,FOODS_3_793_WI_validation,FOODS_3_793,FOODS_3,FOODS,WI_1,WI,185.518183,202.618273,192.550071,207.583760,170.400803,196.209746,140.935944,221.979142,176.230835,186.481146,224.896363,162.884556,199.800729,218.117869,207.186184,223.263294,183.191267,210.845907,151.384038,238.334224,189.136016,200.054118,241.166688,174.597964,214.083274,233.617465,221.822296,238.942828,195.981732,225.482068
864,FOODS_3_799_WI_validation,FOODS_3_799,FOODS_3,FOODS,WI_1,WI,36.125082,44.914834,44.304547,32.281713,20.368734,20.105670,17.964346,19.251587,16.878546,27.969222,27.829177,27.205768,36.115373,44.902763,44.292640,32.273036,20.363259,20.100266,17.959517,19.246413,16.874009,27.961704,27.821696,27.198454,36.105664,44.890691,44.280732,32.264360,20.357784,20.094862
865,FOODS_3_804_WI_validation,FOODS_3_804,FOODS_3,FOODS,WI_1,WI,1501.170245,1771.835588,1997.942158,1867.449419,1520.687590,1404.426909,1417.964755,1365.704239,1306.671818,1205.790337,1536.703717,1396.075502,1748.710509,2060.047609,2318.586924,2163.196422,1758.381150,1621.125533,1633.974755,1571.144937,1500.798846,1382.739090,1759.489274,1596.057255,1996.250774,2348.259630,2639.231690,2458.943424,1996.074710,1837.824158
866,FOODS_3_807_WI_validation,FOODS_3_807,FOODS_3,FOODS,WI_1,WI,196.785125,213.987433,200.517458,217.802498,203.501708,191.162894,210.728757,225.177126,182.115568,224.678739,202.620390,221.377406,168.804932,183.196465,171.314534,185.692516,173.126877,162.270388,178.472785,190.264222,153.509599,188.919039,169.938006,185.183033,140.824739,152.405497,142.111610,153.582534,142.752046,133.377882
